In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.003797292709350586
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 32.023415921783105
200000 52.995931075117866
300000 56.10616580286702
400000 50.45905844628901
500000 51.70791651421263
600000 51.97350019044144
700000 54.955819306151426
800000 58.779835906442756
900000 51.91670681086081
1000000 44.04310895658575
1100000 30.50383022515585
1200000 50.66723490426123
1300000 52.59828106372783
1400000 56.35706408457606
1500000 62.793032490475696
1600000 53.803381705279875
1700000 52.73129345901932
1800000 54.74747640937679
1900000 51.06470492510902
2000000 44.52905173537335
2100000 33.79369466181263
2200000 62.89393901681522
2300000 45.406155022949065
2400000 60.05720240623804
2500000 59.08384981493286
2600000 55.808945805398224
2700000 53.72425458334426
2800000 55.15588912759913
2900000 51.08720671172717
3000000 43.9266485

30100000 78.12089509358263
30200000 77.79905318703426
30300000 33.44178928001931
30400000 77.01151951559852
30500000 78.7915633219911
30600000 74.90662874327606
30700000 74.77994924498331
30800000 70.52163806908175
30900000 69.99360264034061
31000000 54.177235435595705
31100000 41.41467194661217
31200000 73.07061321361176
31300000 74.67737885956477
31400000 71.27538451775222
31500000 72.9635246944192
31600000 74.29322318582467
31700000 75.69291999250112
31800000 42.29595462359387
31900000 69.6568741324226
32000000 79.5337092109007
32100000 73.94964370960871
32200000 73.54983664749716
32300000 35.880583351486365
32400000 72.58805761090451
32500000 75.81162163051282
32600000 74.06873286640071
32700000 69.65149434054149
32800000 70.26636308235706
32900000 65.90977794135075
33000000 55.48686957910473
33100000 40.43469498083554
33200000 68.20706386953766
33300000 66.7278937565241
33400000 68.66595339708022
33500000 72.37819482814204
33600000 72.11599501694668
33700000 71.10350361270086
3380

60500000 50.94020352701948
60600000 48.276700288270206
60700000 63.52851941581932
60800000 63.06740392941018
60900000 63.30355580110392
61000000 56.42685379431734
61100000 35.23457465199326
61200000 62.607776901089736
61300000 64.10325901961114
61400000 64.10571539195516
61500000 63.76156243319716
61600000 32.836500183172056
61700000 65.77631970666182
61800000 75.26530123883364
61900000 68.2696624554497
62000000 75.1317233119068
62100000 79.68850392610452
62200000 72.18897454660004
62300000 74.74320866235959
62400000 74.44613008350858
62500000 59.25820958660612
62600000 40.529602377296385
62700000 68.6485458839505
62800000 67.17330643781355
62900000 66.5613660934573
63000000 56.58881642321287
63100000 34.27221458564281
63200000 67.07302881540902
63300000 66.7025074974277
63400000 66.09774287823154
63500000 59.06850066636823
63600000 39.40378928153909
63700000 73.40920576298302
63800000 73.37489953615284
63900000 76.88295210820016
64000000 77.14103029671759
64100000 72.34497287812104
64

90900000 61.73161714335844
91000000 54.72479443849947
91100000 35.6475258944366
91200000 60.80927412498098
91300000 50.079890881361955
91400000 49.29089493601005
91500000 62.6438325986406
91600000 67.77015315936846
91700000 64.29799011098761
91800000 69.56768096888294
91900000 71.18135370612535
92000000 72.23398389069156
92100000 69.22731664615893
92200000 66.99710926344497
92300000 63.81895758492867
92400000 63.46695910483443
92500000 62.59854917171373
92600000 62.17730493869728
92700000 60.74455579575464
92800000 27.300122416463335
92900000 59.05644382572633
93000000 55.31876481016778
93100000 29.084282705372317
93200000 56.20502086646409
93300000 42.7456702898997
93400000 49.74695391630904
93500000 60.464653385217886
93600000 59.14272494819343
93700000 61.59710942247866
93800000 63.62283984072958
93900000 65.1487271516866
94000000 65.42748044539877
94100000 69.4149631068948
94200000 71.14548037408495
94300000 69.77234160875217
94400000 65.32450360291695
94500000 68.37599896076847
94

120400000 73.01999507465266
120500000 73.39579617490752
120600000 72.0685355534439
120700000 73.97965347138667
120800000 72.22747364929681
120900000 73.10368526345954
121000000 38.249285134198445
121100000 11.416547935100805
121200000 65.5780316785734
121300000 71.68677229695763
121400000 72.43954067070145
121500000 73.90648082081782
121600000 77.8458362755256
121700000 71.71944819446189
121800000 74.38067937137157
121900000 77.4607465694015
122000000 80.27514377325257
122100000 23.25144047655602
122200000 7.127757729161494
122300000 6.939338946855175
122400000 6.846227790801188
122500000 6.413418751927046
122600000 6.399767297715098
122700000 6.7953257261398035
122800000 6.513864470483736
122900000 6.081957202942657
123000000 6.406901503029997
123100000 6.348036477183456
123200000 6.58205748557608
123300000 6.090400742676692
123400000 6.183288559363085
123500000 6.871859432414371
123600000 6.710108840728943
123700000 7.187698552249218
123800000 6.376671811806017
123900000 6.2216168381

...

884700000 2.410382890844049
884800000 3.210198436596389
884900000 1.6813236504696367
885000000 2.780935144305571
885100000 2.261857347799973
885200000 2.9068464835512224
885300000 2.8423317153886067
885400000 2.2170164469707436
885500000 3.065994649863571
885600000 1.8970386685865137
885700000 3.1278453222092115
885800000 1.2620984403972924
885900000 2.852802027748869
886000000 1.65681917633323
886100000 2.7353155388260424
886200000 2.1711912132954456
886300000 2.20650822533308
886400000 3.5466721085525768
886500000 1.9893908147381822
886600000 1.4724587035787198
886700000 0.48574710826655426
886800000 0.5070042941974434
886900000 0.5054695309731695
887000000 0.5010959527576685
887100000 0.4299599652930631
887200000 0.3321201774265038
887300000 0.46746884659139715
887400000 0.5002928953132484
887500000 0.4956967190062582
887600000 0.4413845422399263
887700000 2.32842215853719
887800000 1.9707368431736987
887900000 3.8135722995269363
888000000 1.8255276351927543
888100000 2.713565486258

best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.9154725687353679
909400000 0.4404358025404162
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.33696309885799886
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.31971059565972887
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.3121529345307946
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.6960703649407805
909900000 1.1071781010210124
910000000 1.3562195525346736
910100000 1.325391806

920900000 0.5790009177657067
921000000 0.8265009808586068
921100000 1.2438197716595856
921200000 0.7828200746981681
921300000 0.8021598067949971
921400000 0.8239753008517908
921500000 1.0582843053653141
921600000 1.2514952834630304
921700000 0.5707655771451301
921800000 1.28986031827931
921900000 1.0106891030379759
922000000 0.985816414618196
922100000 0.2647937315666795
922200000 0.15790154816922744
922300000 0.23318542413958973
922400000 0.23219638997329275
922500000 0.23237046149316695
922600000 0.19679331209031078
922700000 0.2449198402657191
922800000 0.22752711777858337
922900000 0.14414425138807363
923000000 0.17262815021731853
923100000 0.21767500775972473
923200000 0.20612645282508799
923300000 0.48028613692912453
923400000 0.916167842738839
923500000 1.39923987355876
923600000 1.2042714914612354
923700000 0.6416923443076431
923800000 0.8392409504749371
923900000 0.8508286787670553
924000000 0.8428696539687898
924100000 0.6555558781677207
924200000 0.584611941452672
924300000 

944700000 0.13533905689598388
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.1028769145026505
944900000 0.07025691379903753
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.08166264564071364
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.09938989476049809
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.09608599477094014
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
type

best so far: 0
type: [3, 14, 1, 3, 1] 126
cases of this type: 378
best so far: 0
type: [3, 14, 3, 1, 1] 126
cases of this type: 126
best so far: 0
type: [3, 21, 1, 1, 2] 126
cases of this type: 336
best so far: 0
type: [3, 21, 1, 2, 1] 126
cases of this type: 168
best so far: 0
type: [3, 21, 2, 1, 1] 126
cases of this type: 84
best so far: 0
type: [3, 42, 1, 1, 1] 126
cases of this type: 42
best so far: 0
type: [6, 1, 1, 1, 21] 126
cases of this type: 194481
955400000 0.004791813743537664
955500000 0.0037063074300328898
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.003440783715975947
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
ca